In [1]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
import multiprocessing as mp
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from itertools import combinations
from sklearn.metrics import roc_auc_score

In [2]:
oof_predictions = pd.read_csv('/Users/haris/Desktop/public_kernal_outputs/best_models_v3/allbest_models_oof_predictions_v3.csv')

In [3]:
oof_predictions.dtypes

target                           int64
bin_te_xgb_oof                 float64
bin_oh_xgb_oof                 float64
te_rgf_oof                     float64
bin_te_lgb_oof                 float64
bin_oh_lgb_oof                 float64
best_nn_oof                    float64
bin_fp_xgb_oof                 float64
oh_mlp_278_oof                 float64
te_xgb_strain_slim_tree_oof    float64
oh_xgb_slim_tree_oof           float64
rgf_new_feat_oof               float64
dtype: object

In [4]:
#helper functions
def auc_to_gini(score):
    """converts AUC into Gini
    """
    return (2 * score) - 1
    
def bold(text_to_bold):
    """
    Bolds given string.
    """
    bold = "\033[1m"
    reset = "\033[0;0m"
    return bold + text_to_bold + reset

In [5]:
target = oof_predictions['target']
oof_predictions.drop('target', axis=1, inplace=True)

In [6]:
oof_names = []
for col in oof_predictions.columns:
    oof_names.append(col)
    
big_list = []
for index in range(0, len(oof_names)):
    print '---'
    print index + 1
    for combo in combinations(oof_names, index + 1):
        new_list =[]
        for combo_item in combo:
            new_list.append(combo_item)
        big_list.append(new_list)
    
print '---'
print len(big_list)
#for item in big_list:
#    print '---'
#    print item

---
1
---
2
---
3
---
4
---
5
---
6
---
7
---
8
---
9
---
10
---
11
---
2047


In [7]:
def calculate_stacker_score(combo):
    # rf = LogisticRegression(random_state = 33)
    df = pd.DataFrame()
    for column_name in combo:
        df[column_name] = oof_predictions[column_name]
        
    #results = cross_val_score(rf, 
    #                          df, 
    #                          y=target,
    #                          cv=5, 
    #                          scoring='roc_auc'
    #                         )
    ranked_oof = (df.rank() / df.shape[0]).mean(axis=1)
    scaler = MinMaxScaler()
    scaled_oof = scaler.fit_transform(ranked_oof.values.reshape(-1, 1))[:,0]
    gini_result = auc_to_gini(roc_auc_score(target, scaled_oof))

    print '----------------'
    #print counter, '/', len(big_list)
    #counter += 1
    print str(combo)
    print bold('CV of stacker:') + ' {:.4f}'.format(gini_result)
    # scores[str(combo_name)] = auc_to_gini(results.mean())
    return str(combo), gini_result

In [8]:
scores = {}
counter = 1
#for combo in big_list:
pool = mp.Pool(processes=8)
results = pool.map(calculate_stacker_score, big_list)
    # print(results)

----------------
['bin_te_xgb_oof']
CV of stacker: 0.2885
----------------
['te_xgb_strain_slim_tree_oof', 'rgf_new_feat_oof']
CV of stacker: 0.2889
----------------
['bin_oh_xgb_oof', 'bin_oh_lgb_oof', 'oh_mlp_278_oof']
CV of stacker: 0.2883
----------------
['te_rgf_oof', 'bin_te_lgb_oof', 'bin_fp_xgb_oof', 'oh_xgb_slim_tree_oof']
CV of stacker: 0.2907
----------------
['bin_te_lgb_oof', 'oh_mlp_278_oof', 'rgf_new_feat_oof']
CV of stacker: 0.2891
----------------
['bin_oh_xgb_oof']
CV of stacker: 0.2886
----------------
----------------
['bin_te_xgb_oof', 'bin_oh_lgb_oof', 'best_nn_oof', 'rgf_new_feat_oof']
['bin_te_xgb_oof', 'bin_oh_xgb_oof', 'best_nn_oof', 'rgf_new_feat_oof']
----------------
CV of stacker: 0.2914
CV of stacker: 0.2914
['bin_oh_xgb_oof', 'bin_te_lgb_oof', 'bin_oh_lgb_oof', 'rgf_new_feat_oof']
CV of stacker: 0.2915
----------------
['oh_xgb_slim_tree_oof', 'rgf_new_feat_oof']
CV of stacker: 0.2888
----------------
['te_rgf_oof']
CV of stacker: 0.2840
---------------

In [9]:
# print results

In [17]:
scores = []
for item in results:
    scores.append(item[1])

scores.sort(reverse=True)
#for item in scores[0:100]:
#    print item
    
for item in results:
    if item[1] > 0.2920:
        print '-------'
        print item[1]
        print item[0]

-------
0.292013170203
['bin_te_xgb_oof', 'bin_oh_lgb_oof', 'best_nn_oof', 'bin_fp_xgb_oof', 'te_xgb_strain_slim_tree_oof', 'rgf_new_feat_oof']
-------
0.292052092877
['bin_te_xgb_oof', 'bin_oh_xgb_oof', 'te_rgf_oof', 'bin_oh_lgb_oof', 'best_nn_oof', 'te_xgb_strain_slim_tree_oof', 'rgf_new_feat_oof']
-------
0.292018780278
['bin_te_xgb_oof', 'bin_oh_xgb_oof', 'bin_te_lgb_oof', 'bin_oh_lgb_oof', 'best_nn_oof', 'te_xgb_strain_slim_tree_oof', 'rgf_new_feat_oof']
-------
0.292021455272
['bin_te_xgb_oof', 'bin_oh_xgb_oof', 'bin_oh_lgb_oof', 'best_nn_oof', 'bin_fp_xgb_oof', 'te_xgb_strain_slim_tree_oof', 'rgf_new_feat_oof']
-------
0.292068744193
['bin_te_xgb_oof', 'te_rgf_oof', 'bin_oh_lgb_oof', 'best_nn_oof', 'bin_fp_xgb_oof', 'te_xgb_strain_slim_tree_oof', 'rgf_new_feat_oof']
-------
0.292033654374
['bin_te_xgb_oof', 'bin_te_lgb_oof', 'bin_oh_lgb_oof', 'best_nn_oof', 'bin_fp_xgb_oof', 'te_xgb_strain_slim_tree_oof', 'rgf_new_feat_oof']
-------
0.292063407387
['bin_te_xgb_oof', 'bin_oh_xgb_

[0.29177675414546345, 0.29176711997318838, 0.29174320084042726, 0.29174103192086509, 0.29172703551573131, 0.29170846417475738, 0.29170001404652046, 0.29169242875832957, 0.29169053548023172, 0.29168712712729805, 0.29167491305217808, 0.29166097912448352, 0.29165843941794845, 0.29165823125637402, 0.29165444402498997, 0.29165209396963032, 0.29164916024787879, 0.29164636672111, 0.29163957600680535, 0.29162741596480779, 0.29162505652450399, 0.29162229164165798, 0.29161853233254109, 0.29161222627659455, 0.29160936140594385, 0.29160648646934173, 0.29160183035318199, 0.2915988968115526, 0.29159359227883086, 0.29158491167639067, 0.29158002761694246, 0.29157324587042477, 0.29157306511532832, 0.29157212870459448, 0.29157001355612788, 0.29156996019511205, 0.29156560493030415, 0.29156549387842756, 0.29156289087173515, 0.29156276181286134, 0.29155986975188886, 0.29155964285790814, 0.2915544464522053, 0.29154785868847855, 0.29154662448696089, 0.29154542426545516, 0.29154452193556679, 0.291536965088389

In [18]:
oof_names

['bin_te_xgb_oof',
 'bin_oh_xgb_oof',
 'te_rgf_oof',
 'bin_te_lgb_oof',
 'bin_oh_lgb_oof',
 'best_nn_oof',
 'bin_fp_xgb_oof',
 'oh_mlp_278_oof',
 'te_xgb_strain_slim_tree_oof',
 'oh_xgb_slim_tree_oof',
 'rgf_new_feat_oof']

In [24]:
counts = {
    'bin_te_xgb_oof': 0,
    'bin_oh_xgb_oof': 0,
    'te_rgf_oof': 0,
    'bin_te_lgb_oof': 0,
    'bin_oh_lgb_oof': 0,
    'best_nn_oof': 0,
    'bin_fp_xgb_oof': 0,
    'oh_mlp_278_oof': 0,
    'te_xgb_strain_slim_tree_oof': 0,
    'oh_xgb_slim_tree_oof': 0,
    'rgf_new_feat_oof': 0
}

In [25]:
for item in results:
    if item[1] > 0.2915:
        for key in counts:
            if key in item[0]:
                counts[key] = counts[key] + 1

In [26]:
for key in counts:
    print key, counts[key]

bin_te_lgb_oof 295
oh_xgb_slim_tree_oof 252
oh_mlp_278_oof 136
best_nn_oof 269
te_rgf_oof 296
te_xgb_strain_slim_tree_oof 329
bin_oh_lgb_oof 358
bin_fp_xgb_oof 308
rgf_new_feat_oof 433
bin_te_xgb_oof 323
bin_oh_xgb_oof 303


In [ ]:
(“[‘bin_te_xgb_oof’,
   ‘bin_oh_xgb_oof’, 
   ‘te_rgf_oof’,
   ‘bin_te_lgb_oof’,
   ‘bin_oh_lgb_oof’,
   ‘best_nn_oof’,
   ‘bin_fp_xgb_oof’,
   ‘te_xgb_strain_slim_tree_oof’,
   ‘rgf_new_feat_oof’]“